Esse programa é para validação dos CNPJ do XML das danfes. Necessário incluir a lista_chaves com os CNPJ/CPF que serão validados.
E colocar as danfes em um diretório separado com os XML das danfes para ele ler, e identificar no pasta_xmls

In [1]:
import  xml.etree.ElementTree

In [ ]:
import os
import xml.etree.ElementTree as ET

#lista
lista_chaves = """
lista dos cnpj's XXXXXXXXXX
""".strip().splitlines()


lista_chaves = set([chave.strip() for chave in lista_chaves if chave.strip().isdigit()])

def extrair_chaves_do_xml(xml_path):
    resultados = []
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        ns = {'ns': root.tag.split('}')[0].strip('{')}

        #chave
        infnfe = root.find('.//ns:infNFe', ns)
        if infnfe is None:
            return []

        chave_acesso = infnfe.attrib.get('Id', '').replace('NFe', '')

        #emitente e destinatário
        emit_cnpj = root.find('.//ns:emit/ns:CNPJ', ns)
        emit_cpf  = root.find('.//ns:emit/ns:CPF', ns)
        dest_cnpj = root.find('.//ns:dest/ns:CNPJ', ns)
        dest_cpf  = root.find('.//ns:dest/ns:CPF', ns)

        pares = {
            'emitente': emit_cnpj or emit_cpf,
            'destinatário': dest_cnpj or dest_cpf
        }

        for tipo, tag in pares.items():
            if tag is not None and tag.text:
                doc = tag.text.strip()
                if doc in lista_chaves:
                    resultados.append((doc, chave_acesso, tipo))

    except Exception as e:
        print(f"Erro ao processar {os.path.basename(xml_path)}: {e}")
    return resultados

pasta_xmls = 'caminho'  
relatorio = []

for arquivo in os.listdir(pasta_xmls):
    if arquivo.lower().endswith('.xml'):
        caminho_arquivo = os.path.join(pasta_xmls, arquivo)
        resultados = extrair_chaves_do_xml(caminho_arquivo)
        relatorio.extend(resultados)


print("\n=== RELATÓRIO: CNPJs/CPFs encontrados com suas Notas ===")
for doc, chave, tipo in sorted(relatorio):
    print(f"{tipo.title():<12} | {doc} | Chave: {chave}")


import csv
nome_arquivo = 'notas_filtradas.csv'
campos = ['chave', 'emitente', 'destinatário']
with open(nome_arquivo, mode='w', newline='', encoding='utf-8') as arquivo_csv:
    escritor = csv.DictWriter(arquivo_csv, fieldnames=campos)
    escritor.writeheader()
    escritor.writerows(resultados)

print(f'\n Relatório salvo em: {nome_arquivo}')




C:\Users\PATRICIASANTOS\AppData\Local\Temp\ipykernel_15252\86019018.py:320: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  'emitente': emit_cnpj or emit_cpf,
C:\Users\PATRICIASANTOS\AppData\Local\Temp\ipykernel_15252\86019018.py:321: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  'destinatário': dest_cnpj or dest_cpf



=== RELATÓRIO: CNPJs/CPFs encontrados com suas Notas ===

✅ Relatório salvo em: notas_filtradas.csv
